In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
pd.options.mode.chained_assignment = None
# import folium
# from folium.plugins import HeatMap

In [68]:
engine = create_engine('postgresql+psycopg2://{user}:{passwd}@{host}:{port}/{db}'.format(
        user='postgres',
        passwd=12345,
        host='localhost',
        port='5432',
        db='c2m2'))

In [2]:
business_df = pd.read_excel('../raw/workers_data.xlsx')
variable_map = pd.read_excel('../data/map_viz_variable.xlsx')
labels_map = pd.read_excel('../data/generated_business_labels_map.xlsx')

In [70]:
business_df.head()

,m_start,m_end,m_today,m_username,m_deviceid,m_subscriberid,m_imei,m_phonenumber,_span_style_color_FF_finances_and_HR_span,i_econ_stop_business,...,__version__,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags,_index
0,2021-04-28T15:14:45.949+05:45,2021-04-28T15:33:20.591+05:45,2021-04-28,username not found,ee.humanitarianresponse.info:efUjl6IoNb0tnOVO,subscriberid not found,ee.humanitarianresponse.info:efUjl6IoNb0tnOVO,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,174444706,510712d3-b5f6-47e7-b3d2-9ae5f49e3b94,2021-04-28T09:47:31,NaN,[],submitted_via_web,NaN,NaN,36
1,2021-05-07T10:59:17.820+05:45,2021-05-07T11:21:20.855+05:45,2021-05-07,username not found,ee.humanitarianresponse.info:ivuI8g0MlCGor6Ft,subscriberid not found,ee.humanitarianresponse.info:ivuI8g0MlCGor6Ft,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,176510702,8ff407ec-6aab-46c8-bb5b-cc48219cc389,2021-05-07T05:36:20,NaN,[],submitted_via_web,NaN,NaN,109
2,2021-04-26T17:28:00.072+05:45,2021-04-26T17:41:14.457+05:45,2021-04-26,username not found,ee.humanitarianresponse.info:0cXJ5kEGM9avnhKH,subscriberid not found,ee.humanitarianresponse.info:0cXJ5kEGM9avnhKH,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,173876161,fd294dd2-9126-4feb-9b54-a0d41843f063,2021-04-26T11:57:05,NaN,[],submitted_via_web,NaN,NaN,8
3,2021-05-01T08:07:51.848+05:45,2021-05-01T08:45:08.379+05:45,2021-05-01,username not found,ee.humanitarianresponse.info:RyUETda2QuZoNbud,subscriberid not found,ee.humanitarianresponse.info:RyUETda2QuZoNbud,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,175156051,76b937cb-67fb-4892-ac32-5ec77e023929,2021-05-01T03:00:10,NaN,[],submitted_via_web,NaN,NaN,69
4,2021-05-16T19:46:28.056+05:45,2021-05-16T20:06:45.761+05:45,2021-05-16,username not found,ee.humanitarianresponse.info:CS5rQbmD3nN55q2y,subscriberid not found,ee.humanitarianresponse.info:CS5rQbmD3nN55q2y,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,178003908,d9cfe125-2298-41e8-bdd8-17e7262ebbdb,2021-05-16T14:21:47,NaN,[],submitted_via_web,NaN,NaN,110


In [71]:
# business_map_df = business_df[['m_name_business', 'm_today', 'm_coodinates']]
business_df = business_df[business_df['m_coodinates']!=' ']

In [72]:
# business_map_df['coordinates'] = business_map_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
# business_map_df.drop('m_coodinates', axis=1, inplace=True)

In [73]:
# business_map_df.head()

In [74]:
selected_variables = variable_map[variable_map['selected']=='yes']
s_select_variables = selected_variables[selected_variables['input_type']=='single-select']['variable'].tolist()
m_select_variables = selected_variables[selected_variables['input_type']=='multi-select']['variable'].tolist()

In [75]:
s_select_df = []
for i in s_select_variables:
    each_var_df = business_df[['m_name_business', 'm_today', 'm_coodinates', 'm_biz_type']]
    each_var_df = each_var_df[each_var_df['m_coodinates']!=' ']
    each_var_df['coordinates'] = each_var_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
    each_var_df.drop('m_coodinates', axis=1, inplace=True)
    each_var_df['variable'] = i
    each_var_df['value'] = business_df[i]
    each_var_df.dropna(inplace=True)
    each_var_df['value'] = each_var_df['value'].astype(int)
    each_var_df['variable_value'] = each_var_df['value'].apply(func=lambda x: i+'__'+str(x))
    percentage_dict = np.round(each_var_df['value'].value_counts()/len(each_var_df)*100, 0).to_dict()
    total_dict = each_var_df['value'].value_counts().to_dict()
    each_var_df['perc_of_total'] = each_var_df['value'].apply(lambda x: percentage_dict[x]).astype(int)
    each_var_df['total'] = each_var_df['value'].apply(lambda x: total_dict[x])
    if len(s_select_df)==0:
        s_select_df = each_var_df
    else:
        s_select_df = pd.concat((s_select_df, each_var_df), axis=0)

In [76]:
s_select_df.head()

,m_name_business,m_today,m_biz_type,coordinates,variable,value,variable_value,perc_of_total,total
0,Shisha Lounge & Bar Pvt Ltd,2021-04-28,2,"(27.7153038584918, 85.3103238891712)",i_econ_stop_business,2,i_econ_stop_business__2,77,84
1,Hotel Vinayak pvt ltd,2021-05-07,2,"(26.98806, 85.89996)",i_econ_stop_business,2,i_econ_stop_business__2,77,84
2,chitwan forest resort,2021-04-26,1,"(27.58148, 84.49332)",i_econ_stop_business,2,i_econ_stop_business__2,77,84
3,Suburb cafe,2021-05-01,1,"(27.684759, 85.312688)",i_econ_stop_business,2,i_econ_stop_business__2,77,84
4,hostel nextdoor,2021-05-16,1,"(27.684944, 85.315922)",i_econ_stop_business,2,i_econ_stop_business__2,77,84


In [77]:
m_select_variables = selected_variables[selected_variables['input_type']=='multi-select']['variable'].tolist()
m_select_variables = labels_map.set_index('variable').transpose()[m_select_variables]
m_select_variables = m_select_variables.transpose().reset_index()
m_select_variables = m_select_variables['variableLabel'].tolist()

In [78]:
def set_value(values, variable):
    output =[]
    for value in values:
        if value==1:
            output.append(variable.split('__')[1])
        else:
            output.append(np.nan)
    return output
        
m_select_df = []
for i in m_select_variables:
    each_var_df = business_df[['m_name_business', 'm_today', 'm_coodinates', 'm_biz_type']]
    each_var_df = each_var_df[each_var_df['m_coodinates']!=' ']
    each_var_df['coordinates'] = each_var_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
    each_var_df.drop('m_coodinates', axis=1, inplace=True)
    each_var_df['variable'] = i.split('__')[0]
    each_var_df['value'] = business_df[i]
    each_var_df['value'] = set_value(each_var_df['value'], i)
    each_var_df.dropna(inplace=True)
    variable_label = {}
    variable_label[i.split('__')[0]] = list(labels_map[labels_map['variable']==i.split('__')[0]]['variableLabel'])
    asked_total = 0
    for j in business_df[variable_label[i.split('__')[0]]].values:
        if (1 in j) or (0 in j):
            asked_total +=1
    each_var_df['value'] = each_var_df['value'].astype(int)
    each_var_df['variable_value'] = i
    each_var_df['total'] = len(each_var_df)
    each_var_df['perc_of_total'] =(each_var_df['total']/len(each_var_df)).astype(int)
    if len(m_select_df)==0:
        m_select_df = each_var_df
    else:
        m_select_df = pd.concat((m_select_df, each_var_df), axis=0)

In [79]:
map_viz_df = pd.concat((s_select_df, m_select_df), axis=0)

In [80]:
def map_date(date_column):
    output = []
    for date in date_column:
        if date <= pd.to_datetime('2021-04-29'):
            output.append(1)
        elif (date > pd.to_datetime('2021-04-29')) and (date <= pd.to_datetime('2021-05-15')):
            output.append(2)
        elif(date > pd.to_datetime('2021-05-15')) and (date <=pd.to_datetime('2021-06-01')):
            output.append(3)
        elif(date > pd.to_datetime('2021-06-01')):
            output.append(4)
    return output

In [81]:
map_viz_df['m_today'] = pd.to_datetime(map_viz_df['m_today'])
map_viz_df['m_today'] = map_date(map_viz_df['m_today'])
map_viz_df['label_en'] = map_viz_df['variable_value'].apply(func=lambda x: labels_map[labels_map['variableLabel']==x]['label_en'].tolist()[0])
map_viz_df['label_ne'] = map_viz_df['variable_value'].apply(func=lambda x: labels_map[labels_map['variableLabel']==x]['label_ne'].tolist()[0])
map_viz_df['m_biz_type'] = map_viz_df['m_biz_type'].apply(func=lambda x: 'm_biz_type__'+str(x))
map_viz_df['m_biz_type'] = map_viz_df['m_biz_type'].apply(func= lambda x: labels_map[labels_map['variableLabel']==x]['label_en'].tolist()[0])
map_viz_df['latitude'] = map_viz_df['coordinates'].apply(func=lambda x: x[0])
map_viz_df['longitude'] = map_viz_df['coordinates'].apply(func=lambda x: x[1])
map_viz_df['index'] = range(1,len(map_viz_df)+1)

In [82]:
map_viz_df.to_csv('map_viz_df.csv', index=False)

In [45]:
map_viz_df.drop(['variable_value', 'coordinates'], axis=1, inplace=True)

In [46]:
map_viz_df.columns = ['businessname', 'submissiondate', 'businesstype', 'variable', 'value',
       'percoftotal', 'total', 'label_en', 'label_ne', 'latitude',
       'longitude', 'index']

In [61]:
map_viz_df

,businessname,submissiondate,businesstype,variable,value,percoftotal,total,label_en,label_ne,latitude,longitude,index
0,Shisha Lounge & Bar Pvt Ltd,1,Restaurant and Bar,i_econ_stop_business,2,77,84,"Yes, temporarily stopped operations","हो, अस्थायी रूपमा सञ्चालनहरू बन्द गरियो",27.715304,85.310324,1
1,Hotel Vinayak pvt ltd,2,Restaurant and Bar,i_econ_stop_business,2,77,84,"Yes, temporarily stopped operations","हो, अस्थायी रूपमा सञ्चालनहरू बन्द गरियो",26.988060,85.899960,2
2,chitwan forest resort,1,Hotel,i_econ_stop_business,2,77,84,"Yes, temporarily stopped operations","हो, अस्थायी रूपमा सञ्चालनहरू बन्द गरियो",27.581480,84.493320,3
3,Suburb cafe,2,Hotel,i_econ_stop_business,2,77,84,"Yes, temporarily stopped operations","हो, अस्थायी रूपमा सञ्चालनहरू बन्द गरियो",27.684759,85.312688,4
4,hostel nextdoor,3,Hotel,i_econ_stop_business,2,77,84,"Yes, temporarily stopped operations","हो, अस्थायी रूपमा सञ्चालनहरू बन्द गरियो",27.684944,85.315922,5
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Carrot holidays pvt ltd,2,Travel and Tour Operator,o_rcvry_biggest_diffclties,7,1,55,Shortage of cash flow,नगद प्रवाहको अभाव,27.688299,85.332441,1492
101,Hotel Tharu Garden,2,Hotel,o_rcvry_biggest_diffclties,7,1,55,Shortage of cash flow,नगद प्रवाहको अभाव,27.575352,84.501673,1493
103,DBRAND SOLUTION PVT LTD,2,Other,o_rcvry_biggest_diffclties,7,1,55,Shortage of cash flow,नगद प्रवाहको अभाव,27.692401,85.330144,1494
111,Outlandish Nepal Holidays Ltd.,3,Travel and Tour Operator,o_rcvry_biggest_diffclties,7,1,55,Shortage of cash flow,नगद प्रवाहको अभाव,27.674262,85.369165,1495


In [36]:
map_viz_df.to_sql('map_visualization_data',engine, index=False, if_exists='replace')

In [47]:
# base_map = folium.Map(location=[27.7, 85.3240], zoom_start=13)
# HeatMap(data=business_map_df['coordinates'], radius=9, max_zoom=13, ).add_to(base_map)
# # base_map.save('kirana_heatmap.html')
# base_map

# Final Script

In [62]:
class Maps():
    def __init__(self, raw_data, variable_map, labels_map):
        self.raw_data = raw_data
        self.raw_data = self.raw_data[self.raw_data['m_coodinates']!=' ']
        self.variable_map = variable_map
        self.labels_map = labels_map
        self.selected_variables = self.variable_map[self.variable_map['selected']=='yes']
        
    
    def generate_data(self):
        map_viz_df = pd.concat((self.generate_s_select(), self.generate_multiselect()), axis=0)
        map_viz_df['m_today'] = pd.to_datetime(map_viz_df['m_today'])
        map_viz_df['m_today'] = self.map_date(map_viz_df['m_today'])
        map_viz_df['label_en'] = map_viz_df['variable_value'].apply(func=lambda x: self.labels_map[self.labels_map['variableLabel']==x]['label_en'].tolist()[0])
        map_viz_df['label_ne'] = map_viz_df['variable_value'].apply(func=lambda x: self.labels_map[self.labels_map['variableLabel']==x]['label_ne'].tolist()[0])
        map_viz_df['m_biz_type'] = map_viz_df['m_biz_type'].apply(func=lambda x: 'm_biz_type__'+str(x))
        map_viz_df['m_biz_type'] = map_viz_df['m_biz_type'].apply(func= lambda x: self.labels_map[self.labels_map['variableLabel']==x]['label_en'].tolist()[0])
        map_viz_df['latitude'] = map_viz_df['coordinates'].apply(func=lambda x: x[0])
        map_viz_df['longitude'] = map_viz_df['coordinates'].apply(func=lambda x: x[1])
        map_viz_df['index'] = range(1,len(map_viz_df)+1)
        map_viz_df.drop(['variable_value', 'coordinates'], axis=1, inplace=True)
        map_viz_df.columns = ['businessname', 'submissiondate', 'businesstype', 'variable', 'value',
                   'percoftotal', 'total', 'label_en', 'label_ne', 'latitude','longitude', 'index']
        return map_viz_df
    
    
    def generate_s_select(self):
        s_select_variables = self.selected_variables[self.selected_variables['input_type']=='single-select']['variable'].tolist()
        s_select_df = []
        for i in s_select_variables:
            each_var_df = self.raw_data[['m_name_business', 'm_today', 'm_coodinates', 'm_biz_type']]
            each_var_df['coordinates'] = each_var_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
            each_var_df.drop('m_coodinates', axis=1, inplace=True)
            each_var_df['variable'] = i
            each_var_df['value'] = self.raw_data[i]
            each_var_df.dropna(inplace=True)
            each_var_df['value'] = each_var_df['value'].astype(int)
            each_var_df['variable_value'] = each_var_df['value'].apply(func=lambda x: i+'__'+str(x))
            percentage_dict = np.round(each_var_df['value'].value_counts()/len(each_var_df)*100, 0).to_dict()
            total_dict = each_var_df['value'].value_counts().to_dict()
            each_var_df['perc_of_total'] = each_var_df['value'].apply(lambda x: percentage_dict[x]).astype(int)
            each_var_df['total'] = each_var_df['value'].apply(lambda x: total_dict[x])
            if len(s_select_df)==0:
                s_select_df = each_var_df
            else:
                s_select_df = pd.concat((s_select_df, each_var_df), axis=0)
        return s_select_df
    
    def generate_multiselect(self):
        m_select_df = []
        m_select_variables = self.selected_variables[self.selected_variables['input_type']=='multi-select']['variable'].tolist()
        m_select_variables = self.labels_map.set_index('variable').transpose()[m_select_variables]
        m_select_variables = m_select_variables.transpose().reset_index()
        m_select_variables = m_select_variables['variableLabel'].tolist()
        for i in m_select_variables:
            each_var_df = self.raw_data[['m_name_business', 'm_today', 'm_coodinates', 'm_biz_type']]
            each_var_df['coordinates'] = each_var_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
            each_var_df.drop('m_coodinates', axis=1, inplace=True)
            each_var_df['variable'] = i.split('__')[0]
            each_var_df['value'] = self.raw_data[i]
            each_var_df['value'] = self.set_value(each_var_df['value'], i)
            each_var_df.dropna(inplace=True)
            variable_label = {}
            variable_label[i.split('__')[0]] = list(self.labels_map[self.labels_map['variable']==i.split('__')[0]]['variableLabel'])
            asked_total = 0
            for j in self.raw_data[variable_label[i.split('__')[0]]].values:
                if (1 in j) or (0 in j):
                    asked_total +=1
            each_var_df['value'] = each_var_df['value'].astype(int)
            each_var_df['variable_value'] = i
            each_var_df['total'] = len(each_var_df)
            each_var_df['perc_of_total'] =(each_var_df['total']/len(each_var_df)).astype(int)
            if len(m_select_df)==0:
                m_select_df = each_var_df
            else:
                m_select_df = pd.concat((m_select_df, each_var_df), axis=0)
        return m_select_df
                
    def set_value(self, values, variable):
        output =[]
        for value in values:
            if value==1:
                output.append(variable.split('__')[1])
            else:
                output.append(np.nan)
        return output
        
    
    def map_date(self, date_column):
        output = []
        for date in date_column:
            if date <= pd.to_datetime('2021-04-29'):
                output.append(1)
            elif (date > pd.to_datetime('2021-04-29')) and (date <= pd.to_datetime('2021-05-15')):
                output.append(2)
            elif(date > pd.to_datetime('2021-05-15')) and (date <=pd.to_datetime('2021-06-01')):
                output.append(3)
            elif(date > pd.to_datetime('2021-06-01')):
                output.append(4)
        return output

In [63]:
business_df = pd.read_excel('../raw/business_data.xlsx')
variable_map = pd.read_excel('../data/map_viz_variable.xlsx')
labels_map = pd.read_excel('../data/generated_business_labels_map.xlsx')

In [64]:
maps = Maps(raw_data=business_df, variable_map=variable_map, labels_map=labels_map)

In [66]:
maps.generate_data().to_csv('map_viz_df.csv', index=False)

In [46]:
nep_map = folium.Map(location=[28.5, 84], zoom_start=7)
incidents = folium.plugins.MarkerCluster().add_to(nep_map)
for lat, lon in business_map_df['coordinates']:
    folium.Marker(
    [lat, lon],
    icon=None
    ).add_to(incidents)
# nep_map.save('distribution.html')
nep_map

In [10]:
import psycopg2
from sqlalchemy import create_engine
engine = create_engine('postgresql+psycopg2://{user}:{passwd}@{host}:{port}/{db}'.format(
        user='postgres',
        passwd=12345,
        host='localhost',
        port='5432',
        db='c2m2'))

In [19]:
pd.read_csv('../business_bivariate_stats.csv')

,xvariable,xvalue,xlabel_en,xlabel_ne,yvariable,yvalue,ylabel_en,ylabel_ne,total,percoftotal,variablegroup,index
0,m_biz_years_in_operation,1,Less than a year,एक बर्ष भन्दा कम,i_covid_effect_business,10,Other,अन्य,0,0.000000,impact,1
1,m_biz_years_in_operation,1,Less than a year,एक बर्ष भन्दा कम,i_covid_effect_business,5,Supply chain disruption,आपूर्ति श्रृंखला अवरोध,0,0.000000,impact,2
2,m_biz_years_in_operation,1,Less than a year,एक बर्ष भन्दा कम,i_covid_effect_business,2,Increased COVID-19 cases in our area,हाम्रो क्षेत्रमा कोभिड-१९ केसहरू बढ्यो,1,0.008929,impact,3
3,m_biz_years_in_operation,1,Less than a year,एक बर्ष भन्दा कम,i_covid_effect_business,9,Lack of adequate financial resources,पर्याप्त वित्तीय स्रोतहरूको अभाव,0,0.000000,impact,4
4,m_biz_years_in_operation,1,Less than a year,एक बर्ष भन्दा कम,i_covid_effect_business,1,Reduced volume of business,व्यापारको मात्रा कम गरियो,1,0.008929,impact,5
...,...,...,...,...,...,...,...,...,...,...,...,...
3187,b_n_emplyes_pre_covid,1,Upto 10 people,१० जना सम्म,o_econ_impact_wrkfrc_chng_21_v_19,4,Workforce size will be 75% of 2019,कार्यबल आकार २०१९ को ७५% हुनेछ,1,0.010753,outlook,3188
3188,b_n_emplyes_pre_covid,1,Upto 10 people,१० जना सम्म,o_econ_impact_wrkfrc_chng_21_v_19,5,Workforce size will be similar to that of 2019,कार्यबल आकार २०१९ को जस्तै मिल्दो हुनेछ,5,0.053763,outlook,3189
3189,b_n_emplyes_pre_covid,1,Upto 10 people,१० जना सम्म,o_econ_impact_wrkfrc_chng_21_v_19,3,Workforce size will be 50% of 2019,कार्यबल आकार २०१९ को ५०% हुनेछ,4,0.043011,outlook,3190
3190,b_n_emplyes_pre_covid,1,Upto 10 people,१० जना सम्म,o_econ_impact_wrkfrc_chng_21_v_19,1,None (or negligible),कुनै पनि होइन (वा नगण्य),20,0.215054,outlook,3191


In [15]:
business_df['m_today'] = pd.to_datetime(business_df['m_today'])

In [11]:
def map_date(date_column):
        output = []
        for date in date_column:
            if date <= pd.to_datetime('2021-04-29'):
                output.append(1)
            elif (date > pd.to_datetime('2021-04-29')) and (date <= pd.to_datetime('2021-05-03')):
                output.append(2)
            elif (date > pd.to_datetime('2021-05-03')) and (date <= pd.to_datetime('2021-05-05')):
                output.append(3)
            elif(date > pd.to_datetime('2021-05-05')) and (date <=pd.to_datetime('2021-05-12')):
                output.append(4)
            elif(date > pd.to_datetime('2021-05-12')) and (date <=pd.to_datetime('2021-05-27')):
                output.append(5)
            elif(date > pd.to_datetime('2021-05-27')) and (date <=pd.to_datetime('2021-06-01')):
                output.append(6)
            elif(date > pd.to_datetime('2021-06-01')) and (date <=pd.to_datetime('2021-06-03')):
                output.append(7)
            elif(date > pd.to_datetime('2021-06-03')) and (date <=pd.to_datetime('2021-06-14')):
                output.append(8)
            elif(date > pd.to_datetime('2021-06-14')) and (date <=pd.to_datetime('2021-06-17')):
                output.append(9)
            elif(date > pd.to_datetime('2021-06-17')) and (date <=pd.to_datetime('2021-06-21')):
                output.append(10)
            elif(date > pd.to_datetime('2021-06-21')) and (date <=pd.to_datetime('2021-06-29')):
                output.append(11)
            elif(date > pd.to_datetime('2021-06-29')) and (date <=pd.to_datetime('2021-07-02')):
                output.append(12)
            elif(date > pd.to_datetime('2021-07-02')) and (date <=pd.to_datetime('2021-07-05')):
                output.append(13)
            elif(date > pd.to_datetime('2021-07-05')) and (date <=pd.to_datetime('2021-07-15')):
                output.append(14)
        return output

In [12]:
business_df['m_today'] = map_date(business_df['m_today'])

In [13]:
business_df['m_today'].value_counts()

1    53
2    48
3     4
4     4
5     2
6     1
Name: m_today, dtype: int64

In [17]:
x = business_df[business_df['m_today']<= pd.to_datetime('2021-04-29')]

In [24]:
business_df[business_df['m_today']<= pd.to_datetime('2021-05-03')]

,m_start,m_end,m_today,m_username,m_deviceid,m_subscriberid,m_imei,m_phonenumber,_span_style_color_FF_finances_and_HR_span,i_econ_stop_business,...,__version__,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags,_index
0,2021-04-28T15:14:45.949+05:45,2021-04-28T15:33:20.591+05:45,2021-04-28,username not found,ee.humanitarianresponse.info:efUjl6IoNb0tnOVO,subscriberid not found,ee.humanitarianresponse.info:efUjl6IoNb0tnOVO,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,174444706,510712d3-b5f6-47e7-b3d2-9ae5f49e3b94,2021-04-28T09:47:31,NaN,[],submitted_via_web,NaN,NaN,36
2,2021-04-26T17:28:00.072+05:45,2021-04-26T17:41:14.457+05:45,2021-04-26,username not found,ee.humanitarianresponse.info:0cXJ5kEGM9avnhKH,subscriberid not found,ee.humanitarianresponse.info:0cXJ5kEGM9avnhKH,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,173876161,fd294dd2-9126-4feb-9b54-a0d41843f063,2021-04-26T11:57:05,NaN,[],submitted_via_web,NaN,NaN,8
3,2021-05-01T08:07:51.848+05:45,2021-05-01T08:45:08.379+05:45,2021-05-01,username not found,ee.humanitarianresponse.info:RyUETda2QuZoNbud,subscriberid not found,ee.humanitarianresponse.info:RyUETda2QuZoNbud,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,175156051,76b937cb-67fb-4892-ac32-5ec77e023929,2021-05-01T03:00:10,NaN,[],submitted_via_web,NaN,NaN,69
5,2021-04-27T20:59:52.289+05:45,2021-04-27T21:23:41.138+05:45,2021-04-27,username not found,ee.humanitarianresponse.info:i1dsoIy9wD7AfDmQ,subscriberid not found,ee.humanitarianresponse.info:i1dsoIy9wD7AfDmQ,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,174239755,adcea1f7-c849-4f2f-ad69-381c33fdfc48,2021-04-27T15:38:44,NaN,[],submitted_via_web,NaN,NaN,21
6,2021-04-30T13:21:49.897+05:45,2021-04-30T13:41:54.558+05:45,2021-04-30,username not found,ee.humanitarianresponse.info:eaYEAjRM9EzxZg67,subscriberid not found,ee.humanitarianresponse.info:eaYEAjRM9EzxZg67,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,174968501,c1907622-de73-4793-806f-e0b75732681d,2021-04-30T07:56:59,NaN,[],submitted_via_web,NaN,NaN,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2021-05-01T20:45:17.160+05:45,2021-05-01T21:55:57.592+05:45,2021-05-01,username not found,ee.humanitarianresponse.info:toXtc2l2YgCN0UqM,subscriberid not found,ee.humanitarianresponse.info:toXtc2l2YgCN0UqM,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,175250793,3ad22e5c-40e2-499d-9d92-34e3852f2fec,2021-05-01T16:10:58,NaN,[],submitted_via_web,NaN,NaN,95
105,2021-05-01T22:52:09.008+05:45,2021-05-01T23:17:16.106+05:45,2021-05-01,username not found,ee.humanitarianresponse.info:uA78k0s4fXLaldqf,subscriberid not found,ee.humanitarianresponse.info:uA78k0s4fXLaldqf,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,175263617,848fd539-b2ac-4070-9996-c801fd846196,2021-05-01T17:32:18,NaN,[],submitted_via_web,NaN,NaN,96
106,2021-05-02T08:13:19.103+05:45,2021-05-02T08:25:35.983+05:45,2021-05-02,username not found,ee.humanitarianresponse.info:Z08Kd27JDyTkeNvb,subscriberid not found,ee.humanitarianresponse.info:Z08Kd27JDyTkeNvb,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,175296075,8c0ed6ca-dcdf-4c5f-ac39-6ac29e9f36af,2021-05-02T02:40:37,NaN,[],submitted_via_web,NaN,NaN,98
107,2021-05-02T12:14:00.358+05:45,2021-05-02T12:31:34.961+05:45,2021-05-02,username not found,ee.humanitarianresponse.info:ZhiID73g0BiIx2dc,subscriberid not found,ee.humanitarianresponse.info:ZhiID73g0BiIx2dc,phonenumber not found,NaN,3,...,vwGwixFVQLjhFdchggLZtd,175313907,a68e2179-c02a-40f0-824b-8023e905b87f,2021-05-02T06:46:54,NaN,[],submitted_via_web,NaN,NaN,100


In [9]:
x = pd.read_csv('../map_visualization_data.csv')

In [10]:
x['variable'].unique()

array(['i_econ_stop_business', 'i_wrkfrc_size_chng_2020_v_2019',
       'i_fin_revenue_chng_2020_v_2019', 'o_do_u_know_of_gov_schemes',
       'o_econ_impact_revenue_chng_21_v_19',
       'o_econ_impact_wrkfrc_chng_21_v_19', 'p_hlth_hhs_measures',
       'p_hlth_safety_measures', 'o_expectd_problms_next_6_mnths',
       'o_rcvry_biggest_diffclties'], dtype=object)

In [11]:
x['label_en'].unique()

array(['Temporarily shutdown during the pandemic',
       'Permanently shutdown during the pandemic', 'Never shutdown',
       'Workforce size became zero', 'Workforce size became 25% of 2019',
       'Workforce size became 50% of 2019',
       'Workforce size became 75% of 2019',
       'Workforce size remained the same',
       'Workforce size increased compared to 2019',
       'Revenue stopped completely', 'Revenue became 25% of 2019',
       'Revenue became 50% of 2019', 'Revenue became 75% of 2019',
       'Revenue remained the same', 'Not aware of any government schemes',
       'Aware about government schemes but haven’t used any',
       'Have used one or more government schemes',
       'Revenue is expected to be higher than 2019',
       'Revenue is expected to be similar to 2019’s revenue',
       'Revenue is expected to be 75% of 2019',
       'Revenue is expected to be 50% of 2019',
       'Revenue is expected to be 25% of 2019', 'None (or negligible)',
       'Workforce 

In [7]:
pd.to_datetime(business_df['m_today']).min()

Timestamp('2021-04-15 00:00:00')

In [8]:
x = pd.DataFrame({'a':[1,2,3], 'b':['a', 'b', 'c']})
x['c'] = 'study of '+x['a'].astype('str') +' '+ x['b']

In [9]:
x

,a,b,c
0,1,a,study of 1 a
1,2,b,study of 2 b
2,3,c,study of 3 c
